## Infinity Universe

[Rules.](https://infinitythewiki.com/Rolls#Face_to_Face_Rolls)

[Question on GitHub.](https://github.com/HighDiceRoller/icepool/issues/100)

In [1]:
%pip install icepool

import icepool
from icepool import d20

class InfinityUniverseEvaluator(icepool.MultisetEvaluator):
    def __init__(self, a_sv, b_sv):
        self.a_sv = a_sv
        self.b_sv = b_sv
        
    # Note that outcomes are seen in ascending order by default.
    def next_state(self, state, outcome, a_count, b_count):
        # Initial state is all zeros.
        a_crit, a_success, b_crit, b_success = state or (0, 0, 0, 0)
        
        # First, accumulate scores.
        if outcome < self.a_sv:
            a_success += a_count
        elif outcome == self.a_sv:
            a_crit += a_count
        if outcome < self.b_sv:
            b_success += b_count
        elif outcome == self.b_sv:
            b_crit += b_count

        # Then, cancel the other side's current and previous successes,
        # which must all have been equal or less than the current outcome.
        # Crits continue to cancel future (higher) successes as well.
        if a_crit or (a_count > 0 and outcome <= self.a_sv):
            b_success = 0
        if b_crit or (b_count > 0 and outcome <= self.b_sv):
            a_success = 0
        
        # Finally, cancel all crits if both sides scored any crit.
        if a_crit > 0 and b_crit > 0:
            # Note that successes were already cancelled above.
            # Also, no more outcomes will matter since
            # all remaining outcomes are above SV.
            a_crit = 0
            b_crit = 0
        return a_crit, a_success, b_crit, b_success
    
print(InfinityUniverseEvaluator(a_sv=12, b_sv=10).evaluate(d20.pool(3), d20.pool(2)))

Die with denominator 3200000

| Outcome[0] | Outcome[1] | Outcome[2] | Outcome[3] | Quantity | Probability |
|-----------:|-----------:|-----------:|-----------:|---------:|------------:|
|          0 |          0 |          0 |          0 |   232328 |   7.260250% |
|          0 |          0 |          0 |          1 |   460752 |  14.398500% |
|          0 |          0 |          0 |          2 |   106152 |   3.317250% |
|          0 |          0 |          1 |          0 |   225218 |   7.038063% |
|          0 |          0 |          1 |          1 |    35424 |   1.107000% |
|          0 |          0 |          2 |          0 |     6859 |   0.214344% |
|          0 |          1 |          0 |          0 |   891834 |  27.869813% |
|          0 |          2 |          0 |          0 |   623112 |  19.472250% |
|          0 |          3 |          0 |          0 |   206420 |   6.450625% |
|          1 |          0 |          0 |          0 |   166107 |   5.190844% |
|          1 |        

### Follow-up question: success values above 20

[Question on GitHub.](https://github.com/HighDiceRoller/icepool/issues/101)

For success values above 20, the success value is treated as 20, with each excess point being treated as a bonus to the d20 roll. Rolls above 20 are treated as 20 (i.e. a critical).

In [2]:
# This can be done by modifying the die that goes into the pool.
# Example: a 21 SV versus a 20 SV with one die each.
from icepool import d20, lowest
print(InfinityUniverseEvaluator(a_sv=20, b_sv=20).evaluate(lowest(d20+1, 20).pool(1), d20.pool(1)))

Die with denominator 400

| Outcome[0] | Outcome[1] | Outcome[2] | Outcome[3] | Quantity | Probability |
|-----------:|-----------:|-----------:|-----------:|---------:|------------:|
|          0 |          0 |          0 |          0 |       20 |   5.000000% |
|          0 |          0 |          0 |          1 |      153 |  38.250000% |
|          0 |          0 |          1 |          0 |       18 |   4.500000% |
|          0 |          1 |          0 |          0 |      171 |  42.750000% |
|          1 |          0 |          0 |          0 |       38 |   9.500000% |


